# Загрузка необходимых библиотек

In [1]:
import requests
from fake_useragent import UserAgent
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup
from IPython.display import clear_output
from fake_useragent import UserAgent
import json
from collections import Counter

import time
import datetime
import pandas as pd
import pickle


from tqdm import tqdm
from joblib import Parallel, delayed

import numpy as np
import pandas as pd
# from openpyxl import load_workbook
import datetime
import os

import csv

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker

from tqdm import tqdm as tqdm_notebook

from collections import defaultdict

import pymorphy3

# Cоздание csv-файла с диапазоном дат (с 01.01.2014 по нынешний месяц), в который записываются новостные показатели

In [2]:
date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
range_date = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
pd.DataFrame(range_date, columns = ['time_series']).to_csv('news_indices.csv', index=None)

# Основные функции

In [3]:
m = pymorphy3.MorphAnalyzer()
def pymorphy_lemma(text='The text must be passed to the function as a string!'):
    text = re.sub(r'[^а-яёА-Я]+', ' ', text)
    tokens = text.lower().split()   
    lemmas = []

    for word in tokens:
        try:
            lemma = m.parse(word)[0].normal_form
            lemmas.append(lemma)
        except:
            continue
    return ', '.join(lemmas)


# набор экономических кризисных слов
econom_forec = '''ставка, экономика, инфляция, уровень, снижение, экономический, прогноз, ключевой, глава, набиуллин,\
 считать, регулятор, повышение, заявить, сказать, решение, ситуация, эльвира, оценка, снижение, активность, рецессия,\
 процентная, процент, безработица, стагнация, спад, замедлить, замедлиться, замедляться, износ, перенос, остановка, приостановка, остановить, негатив,\
 негативный, замедление'''.split(', ')

econom_forec = [pymorphy_lemma(i) for i in econom_forec]

In [4]:
# тональный словарь русского языка КартаСловСент
df = pd.read_csv('kartaslovsent.csv', sep = ';')

# словарик негатива
terms = df[df.tag == 'NGTV'][['term','value']]
negative_dict = dict(zip(terms.term.values, terms.value.values))

# словарик позитива 
terms = df[df.tag == 'PSTV'][['term','value']]
positive_dict = dict(zip(terms.term.values, terms.value.values))

drop = """вирус
инфекция
заразиться
заболевание
заражение
умереть
карантин
заболеть
заразить
эпидемия
пневмония
болезнь
инфекционный
карантинный
изоляция
грипп
осложнение
госпитализация
вирусный
обострение
заболеваемость"""

for item in drop.split():
    del negative_dict[item]

def neg_ind(w):
    scores = [negative_dict.get(item, 0) for item in w]
    return sum(scores)

def pos_ind(w):
    scores = [positive_dict.get(item, 0) for item in w]
    return np.mean(scores)

In [5]:
def index_make(df_term):
    corr_matrix = df_term.corr()
    w = np.array(corr_matrix.sum()/corr_matrix.sum().sum())
    print(w)
    index = (np.array(df_term).T*w.reshape(len(w),1)).sum(axis = 0)
    return index

def min_max_scaler(df, col):
    mx = df[col].max()
    mn = df[col].min()
    df[col] = 100*(df[col] - mn)/(mx - mn)
    pass

In [6]:
def write_to_csv(df_index, indicator_name):
    with open('news_indices.csv', 'r') as file:
        reader = csv.reader(file)
        data = list(reader)

    data[0].append(indicator_name)

    for i in range(1, len(data)):
        data[i].append(df_index[indicator_name].to_list()[i - 1])

    with open('news_indices.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

# Индекс промышленного производства

In [9]:
def main_prom_proiz():  
    prom_proiz = ['промышленность', 'добыча', 'ископаемый', 'производство', 'обрабатывающий', 'электричество', 
                    'газ', 'энергия', 'вода', 'товар', 'услуга']
    
    krizis = set(econom_forec + prom_proiz)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_prom_proiz.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]         
        category = item.get('category', 'not_category')   
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)
        exist_token = tokens & krizis

        for tok in exist_token:
            words_dct[obj][tok] += 1


    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')
    
    usl = (df.source == 'tass')
    df_cur = df[usl]
    krizis_words = df_cur.columns[3:] 
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values

    df_agg = df_cur.groupby('date')[krizis_words].sum().sort_index()

    df_index = pd.DataFrame( )
    df_index['krizis_corr'] = index_make(df_agg[list(krizis_words)])/1000 
    min_max_scaler(df_index, 'krizis_corr')

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['prom_proiz']

    write_to_csv(df_index, 'prom_proiz')
        
    return(df_index)

# Импорт, млн долл США

In [10]:
def main_import():  
    importt = ['экспорт', 'импорт']
    
    krizis = set(econom_forec + importt)
 
    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_export_import.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]         # дата без времени
        category = item.get('category', 'not_category')    # категория
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)

        exist_token = tokens & krizis

        pos_title = pos_ind(wds_title)
        pos_text = pos_ind(wds_text)

        words_dct[obj]['title_pos'] +=  pos_title
        words_dct[obj]['text_pos']  +=  pos_text

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')

    usl = (df.source == 'tass')

    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values

    df_agg = df_cur.groupby('date')[['title_pos', 'text_pos']].mean().sort_index()

    df_index = pd.DataFrame( )
    df_index['pozitive'] = df_agg['title_pos'] + df_agg['text_pos']
    min_max_scaler(df_index, 'pozitive')

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['import']
    
    write_to_csv(df_index, 'import')
        
    return(df_index)

# Импорт, % накопленным итогом год к году

In [11]:
def main_import_percent():  
    importt = ['экспорт', 'импорт']
    
    krizis = set(econom_forec + importt)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_export_import.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]         # дата без времени
        category = item.get('category', 'not_category')    # категория
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)

        exist_token = tokens & krizis

        neg_title = neg_ind(wds_title)
        neg_text = neg_ind(wds_text)

        pos_title = pos_ind(wds_title)
        pos_text = pos_ind(wds_text)

        words_dct[obj]['title_neg'] += neg_title
        words_dct[obj]['text_neg']  += neg_text

        words_dct[obj]['title_pos'] +=  pos_title
        words_dct[obj]['text_pos']  +=  pos_text


    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')

    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values

    df_agg = df_cur.groupby('date')[['title_neg', 'text_neg', 'title_pos', 'text_pos']].mean().sort_index()

    
    df_index = pd.DataFrame( )
    df_index['diff'] =  df_agg['title_neg'] + df_agg['text_neg'] + \
                        df_agg['title_pos'] + df_agg['text_pos']
    min_max_scaler(df_index, 'diff')

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['import_%']
    
    write_to_csv(df_index, 'import_%')
    
    return(df_index)

# Экспорт, млн долл США

In [12]:
def main_export():
    export = ['экспорт', 'импорт']

    krizis = set(econom_forec + export)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_export_import.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]        
        category = item.get('category', 'not_category')
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)

        exist_token = tokens & krizis

        pos_title = pos_ind(wds_title)
        pos_text = pos_ind(wds_text)

        words_dct[obj]['title_pos'] +=  pos_title
        words_dct[obj]['text_pos']  +=  pos_text

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])

    df = df.sort_values('date')
    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values

    df_agg = df_cur.groupby('date')[['title_pos', 'text_pos']].mean().sort_index()

    df_index = pd.DataFrame( )
    df_index['pozitive'] = df_agg['title_pos'] + df_agg['text_pos']
    min_max_scaler(df_index, 'pozitive')

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['export']

    write_to_csv(df_index, 'export')
    
    return(df_index)

# Экспорт, % накопленным итогом год к году

In [13]:
def main_export_percent():  
    export = ['экспорт', 'импорт']

    krizis = set(econom_forec + export)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_export_import.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]     
        category = item.get('category', 'not_category')   
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)

        exist_token = tokens & krizis

        neg_title = neg_ind(wds_title)
        neg_text = neg_ind(wds_text)

        pos_title = pos_ind(wds_title)
        pos_text = pos_ind(wds_text)

        words_dct[obj]['title_neg_len'] += neg_title/(len(wds_title) + 1e-4)
        words_dct[obj]['text_neg_len']  += neg_text/(len(wds_text) + 1e-4)
        words_dct[obj]['title_pos_len'] +=  pos_title/(len(wds_title) + 1e-4)
        words_dct[obj]['text_pos_len']  +=  pos_text/(len(wds_text) + 1e-4)

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')
    
    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values
    df_agg = df_cur.groupby('date')[['title_neg_len', 'text_neg_len', 'title_pos_len', 'text_pos_len']].mean().sort_index()

    
    df_index = pd.DataFrame( )
    df_index['diff_len'] = df_agg['title_neg_len'] + df_agg['text_neg_len'] + \
                       df_agg['title_pos_len'] + df_agg['text_pos_len']
    min_max_scaler(df_index, 'diff_len')

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['export_%']
    
    write_to_csv(df_index, 'export_%')
    
    return(df_index)

# Инвестиции в основной капитал накопленным итогом, млрд руб

In [14]:
def main_invest():  
    invest = ['инвестиции', 'капитал', 'основной', 'пии', 'валовые', 'привлечение']

    krizis = set(econom_forec + invest)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_invest.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]         
        category = item.get('category', 'not_category')   
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)
        exist_token = tokens & krizis

        words_dct[obj]['total_cnt'] += 1
        words_dct[obj]['krzis_cnt'] += (len(exist_token) > 0)

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')

    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values
    df_agg = df_cur.groupby('date')[['krzis_cnt', 'total_cnt']].sum().sort_index()

    df_index = pd.DataFrame( )

    ind = (df_agg['krzis_cnt']/(df_agg['total_cnt'] + 1e-4))
    df_index['krizis_share'] = ind.values
    min_max_scaler(df_index, 'krizis_share')
    
    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['invest']
    
    write_to_csv(df_index, 'invest')

    return(df_index)

# Инвестиции, % накопленным итогом год к году

In [15]:
def main_invest_percent():  
    invest = ['инвестиции', 'капитал', 'основной', 'пии', 'валовые', 'привлечение']

    krizis = set(econom_forec + invest)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv("lemmas/main_lemma_invest.csv").to_dict('records')     

    for item in tqdm_notebook(data):
        dt = item['date'][:10]         
        category = item.get('category', 'not_category')   
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)
        exist_token = tokens & krizis

        pos_title = pos_ind(wds_title)
        pos_text = pos_ind(wds_text)


        words_dct[obj]['title_pos'] +=  pos_title
        words_dct[obj]['text_pos']  +=  pos_text

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')

    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values
    df_agg = df_cur.groupby('date')[['title_pos', 'text_pos']].mean().sort_index()

    df_index = pd.DataFrame( )
    df_index['pozitive'] = df_agg['title_pos'] + df_agg['text_pos']
    min_max_scaler(df_index, 'pozitive')

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['invest_%']
    
    write_to_csv(df_index, 'invest_%')

    return(df_index)

# Инфляция, период с начала года к соответствующему периоду предыдущего года (в среднем за год, в терминологии минэк)

In [16]:
def main_infl():  
    infl = ['инфляция', 'цена', 'цены', 'рост']

    krizis = set(econom_forec + infl)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_infl.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]        
        category = item.get('category', 'not_category')   
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)
        exist_token = tokens & krizis

        neg_title = neg_ind(wds_title)
        neg_text = neg_ind(wds_text)

        pos_title = pos_ind(wds_title)
        pos_text = pos_ind(wds_text)

        words_dct[obj]['title_neg_len'] += neg_title/(len(wds_title) + 1e-4)
        words_dct[obj]['text_neg_len']  += neg_text/(len(wds_text) + 1e-4)

        words_dct[obj]['title_pos_len'] +=  pos_title/(len(wds_title) + 1e-4)
        words_dct[obj]['text_pos_len']  +=  pos_text/(len(wds_text) + 1e-4)

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')

    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values
    df_agg = df_cur.groupby('date')[['title_neg_len', 'text_neg_len', 'title_pos_len', 'text_pos_len']].mean().sort_index()

    df_index = pd.DataFrame( )
    df_index['diff_len'] = df_agg['title_neg_len'] + df_agg['text_neg_len'] + \
                           df_agg['title_pos_len'] + df_agg['text_pos_len']
    min_max_scaler(df_index, 'diff_len')
    
    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['infl']
    
    write_to_csv(df_index, 'infl')
    
    return(df_index)

# Розничный товарооборот

In [17]:
def main_tov_oborot():
    tov_oborot = ['розничный', 'товарооборот', 
                            'продукция', 'розница', 'товар', 'услуга', 'производство', 'торговля']

    krizis = set(econom_forec + tov_oborot)

    words_dct = dict( )

    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_tov_oborot.csv').to_dict('records')


    for item in tqdm_notebook(data):
        dt = item['date'][:10]      
        category = item.get('category', 'not_category')   
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)

        exist_token = tokens & krizis

        pos_title = pos_ind(wds_title)
        pos_text = pos_ind(wds_text)

        words_dct[obj]['title_pos'] +=  pos_title
        words_dct[obj]['text_pos']  +=  pos_text

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')

    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values
    df_agg = df_cur.groupby('date')[['title_pos', 'text_pos']].mean().sort_index()

    df_index = pd.DataFrame( )
    df_index['pozitive'] = df_agg['title_pos'] + df_agg['text_pos']
    min_max_scaler(df_index, 'pozitive')

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['tov_oborot']
    
    write_to_csv(df_index, 'tov_oborot')

    return(df_index)

# Розничный товарооборот, темп роста, %

In [18]:
def main_tov_oborot_percent():  
    tov_oborot = ['розничный', 'товарооборот', 
                        'продукция', 'розница', 'товар', 'услуга', 'производство', 'торговля']

    krizis = set(econom_forec + tov_oborot)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_tov_oborot.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]        
        category = item.get('category', 'not_category')   
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)
        exist_token = tokens & krizis

        words_dct[obj]['total_cnt'] += 1
        words_dct[obj]['krzis_cnt'] += (len(exist_token) > 0)

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')

    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values
    df_agg = df_cur.groupby('date')[['krzis_cnt', 'total_cnt']].sum().sort_index()

    df_index = pd.DataFrame( )
    ind = (df_agg['krzis_cnt']/(df_agg['total_cnt'] + 1e-4))
    df_index['krizis_share'] = ind.values
    min_max_scaler(df_index, 'krizis_share')
    
    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['tov_oborot_%']

    write_to_csv(df_index, 'tov_oborot_%')
    
    return(df_index)

# Уровень безработицы

In [19]:
def main_bezrab():  
    bezrab = ['безработица', 'занятость', 'кадров', 'кадры', 'дефицит'] 

    krizis = set(econom_forec + bezrab)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_bezrab.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]      
        category = item.get('category', 'not_category')   
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)
        exist_token = tokens & krizis
        words_dct[obj]['total_cnt'] += 1.

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')

    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values
    df_agg = df_cur.groupby('date')[['total_cnt']].sum().sort_index()

    df_index = pd.DataFrame( )
    df_index['total_cnt'] = df_agg['total_cnt'].values

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['bezrab']
    
    write_to_csv(df_index, 'bezrab')

    return(df_index)

# Реальные располагаемые денежные доходы

In [20]:
def main_real_den_doh():  
    real_den_doh = ['реальные', 'располагаемые', 'денежные', 'доходы', 'население', 'бедность',
             'прожиточный', 'минимум', 'индексация']

    krizis = set(econom_forec + real_den_doh)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_real_den_doh.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]        
        category = item.get('category', 'not_category')   
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)

        exist_token = tokens & krizis

        neg_title = neg_ind(wds_title)
        neg_text = neg_ind(wds_text)

        pos_title = pos_ind(wds_title)
        pos_text = pos_ind(wds_text)

        words_dct[obj]['title_neg'] += neg_title
        words_dct[obj]['text_neg']  += neg_text

        words_dct[obj]['title_pos'] +=  pos_title
        words_dct[obj]['text_pos']  +=  pos_text

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')
    
    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values
    df_agg = df_cur.groupby('date')[['title_neg', 'text_neg', 'title_pos', 'text_pos']].mean().sort_index()

    df_index = pd.DataFrame( )
    df_index['diff'] =  df_agg['title_neg'] + df_agg['text_neg'] + \
                        df_agg['title_pos'] + df_agg['text_pos']
    min_max_scaler(df_index, 'diff')

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['real_den_doh']
    
    write_to_csv(df_index, 'real_den_doh')

    return(df_index)

# Реальная заработная плата

In [21]:
def main_real_zp():  
    real_zp = ['реальная', 'заработная', 'плата', 'денежные', 'доходы', 'население',
             'бедность', 'прожиточный', 'минимум', 'индексация']

    krizis = set(econom_forec + real_zp)

    words_dct = dict( )
    source = 'tass'
    data = pd.read_csv('lemmas/main_lemma_real_zp.csv').to_dict('records')

    for item in tqdm_notebook(data):
        dt = item['date'][:10]      
        category = item.get('category', 'not_category')  
        obj = (dt, category, source)

        if obj not in words_dct:
            words_dct[obj] = defaultdict(lambda:0)

        wds_title = item['title'].split(', ')
        wds_text = item['text'].split(', ')

        tokens = set(wds_title + wds_text)

        exist_token = tokens & krizis

        neg_title = neg_ind(wds_title)
        neg_text = neg_ind(wds_text)

        pos_title = pos_ind(wds_title)
        pos_text = pos_ind(wds_text)

        words_dct[obj]['title_neg'] += neg_title
        words_dct[obj]['text_neg']  += neg_text

        words_dct[obj]['title_pos'] +=  pos_title
        words_dct[obj]['text_pos']  +=  pos_text

    df = pd.DataFrame(words_dct)
    df = df.T
    df.reset_index(inplace=True)
    df.columns = ['date', 'category', 'source'] + list(df.columns[3:])
    df = df.sort_values('date')

    usl = (df.source == 'tass')
    df_cur = df[usl]
    df_cur.loc[:,'date'] = df_cur['date'].apply(lambda w: w[:7]).values
    df_agg = df_cur.groupby('date')[['title_neg', 'text_neg', 'title_pos', 'text_pos']].mean().sort_index()
    
    df_index = pd.DataFrame( )
    df_index['diff'] =  df_agg['title_neg'] + df_agg['text_neg'] + \
                        df_agg['title_pos'] + df_agg['text_pos']
    min_max_scaler(df_index, 'diff')

    date_next_month = datetime.datetime.now().replace(month = datetime.datetime.now().month + 1) 
    df_index.index = pd.date_range(start='2014-01-01', end=date_next_month, freq="ME")
    
    df_index.columns = ['real_zp']
    
    write_to_csv(df_index, 'real_zp')
    
    return(df_index)

In [26]:
# main_prom_proiz()
# main_import()
# main_import_percent()
# main_export()
# main_export_percent()
# main_invest()
# main_invest_percent()
# main_infl()
# main_tov_oborot()
# main_tov_oborot_percent()
# main_bezrab()
# main_real_den_doh()
# main_real_zp()

In [25]:
pd.read_csv('news_indices.csv')  # infl

,time_series,prom_proiz,import,import_%,export,export_%,invest,invest_%,infl,tov_oborot,tov_oborot_%,bezrab,real_den_doh,real_zp
0,2014-01-31,14.802115,28.129855,87.259862,28.129855,83.009366,99.996150,43.496781,78.891034,13.284125,73.991242,13.0,35.158874,80.612692
1,2014-02-28,15.941899,13.723033,90.763898,13.723033,89.555139,99.997713,60.496691,60.775786,29.593319,76.956491,13.0,76.407802,73.164434
2,2014-03-31,15.999054,27.196495,76.789840,27.196495,72.256817,99.998820,60.494554,41.611619,28.547639,81.569100,6.0,64.765946,32.951974
3,2014-04-30,21.261523,16.752955,80.932265,16.752955,77.079328,100.000000,72.942712,62.706873,29.082326,85.800317,6.0,78.882530,44.153664
4,2014-05-31,23.126883,11.121032,80.506942,11.121032,92.013296,99.996965,58.019479,89.628727,11.534835,70.648070,6.0,86.919195,75.540486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2024-03-31,59.298789,47.696830,63.020780,47.696830,55.442093,99.999517,93.839079,73.634627,57.669372,98.076119,44.0,63.373970,54.593161
123,2024-04-30,67.197497,49.486734,67.584610,49.486734,62.686622,99.999838,73.370972,59.882332,63.286932,98.543509,77.0,80.894674,74.925922
124,2024-05-31,61.824281,47.212940,66.526585,47.212940,58.992485,99.998319,51.227312,66.008292,80.471599,99.291517,44.0,46.154929,69.652731
125,2024-06-30,90.354615,47.023423,66.255875,47.023423,60.241831,49.224799,90.176358,55.169653,88.972791,100.000000,76.0,70.472178,80.139099
